# VK URL Scraper

<a href="https://www.bellingcat.com"><img alt="Bellingcat logo: Discover Bellingcat" src="https://img.shields.io/badge/Discover%20Bellingcat-%20?style=for-the-badge&logo=data%3Aimage%2Fpng%3Bbase64%2CiVBORw0KGgoAAAANSUhEUgAAAA4AAAAYCAYAAADKx8xXAAABhGlDQ1BJQ0MgcHJvZmlsZQAAKJF9kT1Iw0AcxV9TS0UqDnZQEcxQneyiIo6likWwUNoKrTqYXPoFTRqSFBdHwbXg4Mdi1cHFWVcHV0EQ%2FABxdnBSdJES%2F5cUWsR4cNyPd%2Fced%2B8AoVllqtkTA1TNMtKJuJjLr4rBVwQwhhBEDEvM1JOZxSw8x9c9fHy9i%2FIs73N%2Fjn6lYDLAJxLHmG5YxBvEs5uWznmfOMzKkkJ8Tjxp0AWJH7kuu%2FzGueSwwDPDRjY9TxwmFktdLHcxKxsq8QxxRFE1yhdyLiuctzir1Tpr35O%2FMFTQVjJcpzmKBJaQRIo6klFHBVVYiNKqkWIiTftxD%2F%2BI40%2BRSyZXBYwcC6hBheT4wf%2Fgd7dmcXrKTQrFgcCLbX%2BMA8FdoNWw7e9j226dAP5n4Err%2BGtNYO6T9EZHixwBA9vAxXVHk%2FeAyx1g6EmXDMmR%2FDSFYhF4P6NvygODt0Dfmttbex%2BnD0CWulq%2BAQ4OgYkSZa97vLu3u7d%2Fz7T7%2BwHEU3LHAa%2FQ6gAAAAZiS0dEAAAAAAAA%2BUO7fwAAAAlwSFlzAAAuIwAALiMBeKU%2FdgAAAAd0SU1FB%2BgFHwwiMH4odB4AAAAZdEVYdENvbW1lbnQAQ3JlYXRlZCB3aXRoIEdJTVBXgQ4XAAAA50lEQVQ4y82SvWpCQRCFz25ERSJiCNqlUiS1b5AuEEiZIq1NOsGXCKms0wXSp9T6dskDiFikyiPc%2FrMZyf3FXSGQ0%2BzuzPl2ZoeVKgQ0gQ2wBVpVHlcDkjM5V%2FJ5nag6sJ%2FZX%2Bh%2FC7gEhqeAFKf7p1M9aB3b5oN1OomB7g1axUBPBr3GQHODHmOgqUF3MZAzKI2d4LWBV4H%2BMXDuJd1a7Cew1k7SwksaHC4LqNaw7aeX9GWHXkC1G1sTAS17Y3Kk2lnp4wNLiz0DrgLq8qt2MfmSSabAO%2FBBXp26dtrADPjOmN%2BAUdG7B3cE61l5hOZiAAAAAElFTkSuQmCC&logoColor=%23fff&color=%23000"></a><!--
--><a href="https://discord.gg/bellingcat"><img alt="Discord logo: Join our community" src="https://img.shields.io/badge/Join%20our%20community-%20?style=for-the-badge&logo=discord&logoColor=%23fff&color=%235865F2"></a><!--
--><a href="https://github.com/bellingcat/vk-url-scraper"><img alt="Github logo: Explore the code" src="https://img.shields.io/badge/Explore%20the%20code-%20?style=for-the-badge&logo=github&color=%2329903b"></a>

This tool allows you to download all the information and media from one or many VKontakte/ВКонтакте/VK URLs in [JSON](https://developer.mozilla.org/en-US/docs/Learn/JavaScript/Objects/JSON) format, along with all their media.

To use it, you first need to create an account on the platform which typically requires a valid phone number and email.

This notebook will show you how to authenticate into VK with Bellingcat's [vk-url-scraper](https://github.com/bellingcat/vk-url-scraper/) tool and how to then download posts from the social media site, including media such as photos and videos.

- Code and issue management: https://github.com/bellingcat/vk-url-scraper/
- Python package: https://pypi.org/project/vk-url-scraper/
- Documentation: https://vk-url-scraper.readthedocs.io/en/latest/

To run this notebook you need a valid `username/phone` and `password` for vk.com.

Enter these below and they will be used throughout the notebook.

In [ ]:
USER = "your_username_or_phone_here"
PASS = "your_password_here"

### Step 1 - install the python package
This project depends on another tool whose python packaged has not been updated at time of writing, so we have to hack its installation directly from github, and accordingly perform the authentication into VK with some Python code in Step 2.

> NOTE: if you get a `You must restart the runtime in order to use newly installed versions.` pop up on Google Colab, go ahead and click restart. After the restart you can move on to the next cells.

In [13]:
# first install the package (see Note above)
!pip install vk-url-scraper

In [ ]:
# then uninstall the broken dependency and install a fixed one
!pip uninstall vk-api -y
!pip install git+https://github.com/python273/vk_api.git@b99dac0ec2f832a6c4b20bde49869e7229ce4742

In [14]:
# to make sure the installation is successful we can call the help method of the tool
!vk_url_scraper --help

usage: vk_url_scraper [-h] -u USERNAME -p PASSWORD [-t TOKEN] [-d | --download | --no-download]
                      --urls ...

Authenticate and scrape information from vk.com based on a URL or set of URLs.

options:
  -h, --help            show this help message and exit
  -u USERNAME, --username USERNAME
                        username for a valid vk.com account (pass empty if using --token)
  -p PASSWORD, --password PASSWORD
                        password for the valid vk.com account (pass empty if using --token)
  -t TOKEN, --token TOKEN
                        optional token, when passed username/password authentication will not be
                        done - good to avoid captcha issues
  -d, --download, --no-download
                        if set then all photos and videos will be downloaded to folder output/
  --urls ...            must be the last argument: any text with one or more urls to scrape


In [17]:
import json
import time
import vk_api
import os

# Initialize account credentials
accounts = [
    {"user": "+14244027026", "pass": "JamieDOliver567"},
    {"user": "+14245223481", "pass": "RyanLAEmily123?."}
]

# File paths
TRACKING_FILE = "processed_posts.json"
DATA_FILE = "vk_posts.json"

# Load processed post IDs from file, or initialize as empty if the file doesn't exist
if os.path.exists(TRACKING_FILE):
    with open(TRACKING_FILE, "r") as f:
        processed_posts = set(json.load(f))
else:
    processed_posts = set()

# Load existing collected data from file, or initialize as empty if the file doesn't exist
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        collected_data = json.load(f)
else:
    collected_data = []

# Function to save processed post IDs to the tracking file
def save_processed_posts():
    with open(TRACKING_FILE, "w") as f:
        json.dump(list(processed_posts), f)
    print("Processed posts saved successfully.")

# Function to save all collected data to JSON file at the end of each iteration
def save_collected_data():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(collected_data, f, ensure_ascii=False, indent=4)
    print("All collected data saved to vk_posts.json")

# Function to authenticate with VK using the specified credentials
def authenticate(user, password):
    session = vk_api.VkApi(user, password)
    try:
        session.auth()
        print(f"Authenticated successfully with {user}")
    except vk_api.AuthError as error_msg:
        print("Authentication failed:", error_msg)
        return None
    return session

# Start collecting posts
def collect_posts(max_iterations=10):  # Set max_iterations for stopping after a certain number of cycles
    keyword = "Maia Sandu"
    account_index = 0
    iteration_count = 0

    while iteration_count < max_iterations:  # Run until max_iterations is reached
        # Switch between accounts every iteration to avoid hitting the rate limit
        account = accounts[account_index]
        session = authenticate(account["user"], account["pass"])
        if not session:
            print("Retrying with the next account after a delay...")
            time.sleep(60)
            account_index = (account_index + 1) % len(accounts)
            continue

        vk = session.get_api()

        try:
            # Temporary storage for this iteration
            iteration_data = []

            # Search posts containing the keyword
            posts = vk.newsfeed.search(q=keyword, count=200)

            for post in posts['items']:
                post_id = post['id']

                # Skip if post has already been processed
                if post_id in processed_posts:
                    continue

                owner_id = post['owner_id']
                url = f"https://vk.com/wall{owner_id}_{post_id}"

                # Fetching post details
                post_info = vk.wall.getById(posts=f"{owner_id}_{post_id}")[0]

                # Get text content, likes count, and comments count
                text = post_info.get("text", "")
                likes_count = post_info.get("likes", {}).get("count", 0)
                comments_count = post_info.get("comments", {}).get("count", 0)

                # Attempt to fetch comments
                comment_texts = []
                try:
                    comments = vk.wall.getComments(owner_id=owner_id, post_id=post_id, count=100)
                    comment_texts = [comment['text'] for comment in comments['items']]
                except vk_api.exceptions.ApiError as e:
                    print(f"Could not access comments for post {url}: {e}")
                    processed_posts.add(post_id)  # Mark as processed even if comments are denied
                    continue  # Skip posts that deny comment access

                # Extract media (photos and videos)
                photos = []
                videos = []
                attachments = post_info.get("attachments", [])

                for attachment in attachments:
                    if attachment['type'] == 'photo':
                        photo_url = max(attachment['photo']['sizes'], key=lambda size: size['width'])['url']
                        photos.append(photo_url)
                    elif attachment['type'] == 'video':
                        video_id = attachment['video']['id']
                        video_owner_id = attachment['video']['owner_id']
                        try:
                            video_info = vk.video.get(videos=f"{video_owner_id}_{video_id}")['items']
                            if video_info:
                                video_url = video_info[0].get('player', '')
                                videos.append(video_url)
                        except vk_api.exceptions.ApiError as e:
                            print(f"Could not access video for post {url}: {e}")

                # Append details to iteration_data and mark as processed
                post_data = {
                    "url": url,
                    "text": text,
                    "likes_count": likes_count,
                    "comments_count": comments_count,
                    "comments": comment_texts,
                    "photos": photos,
                    "videos": videos
                }
                iteration_data.append(post_data)
                processed_posts.add(post_id)

            # Add iteration_data to collected_data and save all data
            collected_data.extend(iteration_data)
            save_collected_data()  # Save data after each iteration
            save_processed_posts()  # Save processed posts as well

            print(f"Iteration {iteration_count + 1} complete. Data saved.")
            iteration_count += 1  # Increment the iteration count

        except vk_api.exceptions.ApiError as e:
            print("API Error:", e)

        # Switch to the next account for the next iteration
        account_index = (account_index + 1) % len(accounts)

        # Delay before the next request to avoid rate limiting
        time.sleep(60)  # Wait 1 minute between requests

    print("Max iterations reached. Stopping data collection.")

# Start the data collection with a limit on iterations
collect_posts(max_iterations=10)  # Adjust max_iterations as needed


Authenticated successfully with +14244027026
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 1 complete. Data saved.
Authenticated successfully with +14245223481
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 2 complete. Data saved.
Authenticated successfully with +14244027026
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 3 complete. Data saved.
Authenticated successfully with +14245223481
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 4 complete. Data saved.
Authenticated successfully with +14244027026
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 5 complete. Data saved.
Authenticated successfully with +14245223481
All collected data saved to vk_posts.json
Processed posts saved successfully.
Iteration 6 complete. Data saved.
Authenticated successfully with +14244027026
All collected

### Step 2 - login
The next cell contains helper python code needed the first time you login.

In [ ]:
import vk_api

def captcha_handler(captcha):
    key = input(
        f"CAPTCHA DETECTED, please solve it and input the solution. url= {captcha.get_url()} :"
    ).strip()
    return captcha.try_again(key.strip())

The next code cell tries to login with the credentials you provided above.

You may get a `CAPTCHA DETECTED` message and a URL. In that case, go to the URL, solve the captcha and input it by clicking on the text box on the right of the message (different notebook environments have different appearances and in some the textbox is only visible when you click it).

In [ ]:
(vk_api.VkApi(USER,PASS,captcha_handler=captcha_handler)).auth(token_only=True)

A `vk_config.v2.json` file should have been created, it contains your access tokens so don't share it.

When that file is present you don't need to re-run the python code in this section.

### Step 3 - scrape a post
We will scrape a post with both text and images using the VK API: https://vk.com/wall-152947668_126406

This will only work if the `vk_config.v2.json` file is present (see Step 2).

The results will be written to `scraped.json` file, if you want to see it in the console remove the final part of the command `> scraped.json`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!vk_url_scraper --username "{USER}" --password "{PASS}" --urls https://vk.com/wall-152947668_126406 > scraped.json

In [ ]:
# let's read the first lines of scraped.json to make sure it's working
!head scraped.json

Besides the post data, we can automatically `--download` the media.

Let's pass two URLs, one with images and one with videos and download all of them.

You can add as many as you want at once, so long as you separate them by a comma `,`.

The JSON will be written to the console, you can direct it to a JSON file for latter processing by appending `> my_output_filename.json`.

In [ ]:
!vk_url_scraper --username "{USER}" --password "{PASS}" --download --urls "https://vk.com/wall-152947668_126406,https://vk.com/video/@kot_minsk?z=video-28021233_456239018%2Fclub28021233%2Fpl_-28021233"_-2

This tool can be used, for example, to automate the scraping and download of large number of URLs from VK.